# Демонстрация обучения
## Пошаговое обучение модели

In [ ]:
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
from tqdm import tqdm

import config
from dataset import create_dataloaders, get_transforms
from model import get_model
from utils import plot_training_curves

%matplotlib inline

In [ ]:
# Определяем устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Используем устройство: {device}')

In [ ]:
# Загружаем данные
train_loader, val_loader, test_loader = create_dataloaders(config.PLANTVILLAGE_DIR)

In [ ]:
# Создаем модель
model = get_model(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Функция для одной эпохи обучения
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for data, target in tqdm(loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)
    
    return total_loss/len(loader), 100.*correct/total

In [ ]:
# Обучение
train_losses, train_accs = [], []

for epoch in range(3):  # Обучаем 3 эпохи для демонстрации
    loss, acc = train_epoch(model, train_loader, criterion, optimizer)
    train_losses.append(loss)
    train_accs.append(acc)
    print(f'Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {acc:.2f}%')

In [ ]:
# Визуализация результатов
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, 'b-o')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(train_accs, 'r-o')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.grid(True)

plt.tight_layout()
plt.show()